# 数据接口

目前有不少支持Python接口的金融数据库，比如Tushare、AKshare、Baostock、wind等，都可以获得国内股票、基金、期货、利率等数据。


## 1.Tushare社区
公众号上不少文章使用了tushare库获取财经和股票交易数据，当时用的是旧版本（tushare）。Tushare社区目前主要维护新版本：tushare pro，数据更稳定质量更高，可获取沪深股票行情、财务、市场参考等数据，以及指数（含国外股指）、基金、期货、期权、宏观经济、行业经济等财经数据，为金融量化爱好者节省了大量宝贵时间。

此外，近期还增加了新闻联播的文本数据，为文本分析和数据挖掘提供了很好的素材。不过，新版本需要注册获取token才能免费使用，

注册网址： https://tushare.pro/register?reg=218422 。

安装（进入cmd模式）：pip install tushare，或升级：pip install tushare --upgrade。

下面以股票行情数据为例，展示下tushare如何获取数据。股票行情数据以股票行情数据为例，简要介绍如何获取数据。

In [ ]:
#先引入后面分析、可视化等可能用到的库
import tushare as ts

import pandas as pd  
import matplotlib.pyplot as plt
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

#设置token
token='你的token'
#ts.set_token(token)
pro = ts.pro_api(token)

个股行情数据：`pro.stock_basic()`
参数：
* is_hs：是否沪深港通标的，N否、H沪股通、S深股通；
* list_status：上市状态，L上市、D退市、P暂停上市；
* exchange：交易所 SSE上交所，SZSE深交所，HKEX港交所。 

`pro.daily(ts_code= 或 trade_date=)`
* 日行情：daily;
* 周行情：weekly;
* 月行情：monthly

In [ ]:
#获取当前上市的股票代码、简称、注册地、行业、上市时间等数据
basic=pro.stock_basic(list_status='L')
#查看前五行数据
#basic.head(5)

#获取平安银行日行情数据
pa=pro.daily(ts_code='000001.SZ', start_date='20180101',
               end_date='20190106')
#pa.head()

#K线图可视化
from pyecharts import Kline
pa.index=pd.to_datetime(pa.trade_date)
pa=pa.sort_index()
v1=list(pa.loc[:,['open','close','low','high']].values)
t=pa.index
v0=list(t.strftime('%Y%m%d'))
kline = Kline("平安银行K线图",title_text_size=15)
kline.add("", v0, v1,is_datazoom_show=True,
         mark_line=["average"],
         mark_point=["max", "min"],
         mark_point_symbolsize=60,
         mark_line_valuedim=['highest', 'lowest'] )
#kline.render("上证指数图.html")

kline

In [ ]:
#定义获取多只股票函数：
def get_stocks_data(stocklist,start,end):
    all_data={}
    for code in stocklist:
        all_data[code]=pro.daily(ts_code=code,
                 start_date=start, end_date=end)
    return all_data

#保存本地
def save_data(all_data):
    for code,data in all_data.items():
        data.to_csv('c:/zjy/stock_data/'+code+'.csv',
                     header=True, index=False)

stocklist=list(basic.ts_code)[:15]
start=''
end=''
all_data=get_stocks_data(stocklist,start,end)

all_data['000002.SZ'].tail()

#将数据保存到本地
save_data(all_data)

#读取本地文件夹里所有文件
import os
#文件存储路径
file='c:/zjy/stock_data/'
g=os.walk(file)
filenames=[]
for path,d,filelist in g:
    for filename in filelist:
        filenames.append(os.path.join(filename))
print(filenames)

#将读取的数据文件放入一个字典中
df={}
#从文件名中分离出股票代码
code=[name.split('.')[0] for name in filenames]
for i in range(len(filenames)):
    filename=file+filenames[i]
    df[code[i]]=pd.read_csv(filename)

#查看第一只股票前五行数据
#df[code[0]].head()

指数数据：`pro.index_daily(ts_code=)`

In [ ]:
def get_index_data(indexs):
    '''indexs是字典格式'''
    index_data={}
    for name,code in indexs.items():
        df=pro.index_daily(ts_code=code)
        df.index=pd.to_datetime(df.trade_date)   
        index_data[name]=df.sort_index()
    return index_data

#获取常见股票指数行情
indexs={'上证综指': '000001.SH','深证成指': '399001.SZ',
         '沪深300': '000300.SH','创业板指': '399006.SZ',
          '上证50': '000016.SH', '中证500': '000905.SH',
         '中小板指': '399005.SZ','上证180': '000010.SH'}
index_data=get_index_data(indexs)
#index_data['上证综指'].head()

#对股价走势进行可视化分析
subjects =list(index_data.keys())
#每个子图的title
plot_pos = [421,422,423,424,425,426,427,428] 
# 每个子图的位置
new_colors = ['#1f77b4','#ff7f0e', '#2ca02c', 
'#d62728','#9467bd','#8c564b', '#e377c2', 
'#7f7f7f','#bcbd22','#17becf']

fig = plt.figure(figsize=(16,18))
fig.suptitle('A股股指走势',fontsize=18)
for pos in np.arange(len(plot_pos)):       
    ax = fig.add_subplot(plot_pos[pos]) 
    y_data =index_data[subjects[pos]]['close']    
    b = ax.plot(y_data,color=new_colors[pos])   
    ax.set_title(subjects[pos])    
    # 将右上边的两条边颜色设置为空，相当于抹掉这两条边
    ax = plt.gca()  
    ax.spines['right'].set_color('none') 
    ax.spines['top'].set_color('none')    
plt.show()  

## 2.Baostock 证券宝
baostock也是免费、开源的证券数据平台。提供了大量准确、完整的证券历史行情数据、上市公司财务数据等。通过python API获取证券数据信息，可以满足量化交易投资者、数量金融爱好者、计量经济从业者数据需求。返回的数据格式：pandas DataFrame类型，以便于用pandas/NumPy/Matplotlib进行数据分析和可视化。

证券宝链接地址:

http://baostock.com/baostock/index.php/Python_API文档。

安装：进入cmd模式，pip install baostock

In [ ]:
import baostock as bs
#### 登陆系统 ####
lg = bs.login()

#### 获取历史K线数据 ####
# query_history_k_data()
fields= "date,code,open,high,low,close"
rs = bs.query_history_k_data("sh.000001", fields,
    start_date='2000-01-01', end_date='2018-09-07', 
    frequency="d", adjustflag="2") 
#frequency="d"取日k线，adjustflag="3"默认不复权，
#1：后复权；2：前复权

data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
result.index=pd.to_datetime(result.date)
#### 结果集输出到csv文件 ####
#result.to_csv("c:/zjy/history_k_data.csv", 
#        encoding="gbk", index=False)
result.head()
#### 登出系统 ####
#bs.logout()

result.info()

#将某些object转化numeric
result=result.apply(pd.to_numeric, errors='ignore')
result.info()

result.close.plot(figsize=(16,8))
ax = plt.gca()  
ax.spines['right'].set_color('none') 
ax.spines['top'].set_color('none')    
plt.show() 

## 3.AKshare

### 什么是AKshare？
先简单介绍下AKShare，内容转自官网。

AKShare 是基于 Python 的开源金融数据接口库，目的是实现对股票、期货、期权、基金、债券、外汇等金融产品和另类数据从数据采集，数据清洗到数据下载的工具，满足金融数据科学家、数据科学爱好者在数据获取方面的需求。

它的特点是利用 AKShare 获取的是基于可信任数据源发布的原始数据，广大数据科学家可以利用原始数据进行再加工，从而得出科学的结论。

代码语法符合 PEP8 规范, 数据接口的命名统一;
最佳支持 Python 3.7.5 及其以上版本;
提供最佳的文档支持, 每个数据接口提供详细的说明和示例, 只需要复制粘贴就可以下载数据;
持续维护由于目标网页变化而导致的部分数据接口运行异常问题;
持续更新财经数据接口, 同时优化源代码;
如何安装AKShare？

> pip install akshare

AKShare作为python库可以直接通过pip安装，十分快捷。

### 如何使用AKShare？
首先要导入AKShare库，然后通过接口函数，获取相应的数据。

最好是在jupyter notebook或者lab环境中来操作，可以很方便地查看和分析数据。

以获取A股公司列表数据为例：

In [ ]:
import akshare as ak
stock_info_a_code_name_df = ak.stock_info_a_code_name()
stock_info_a_code_name_df

目前AKshare数据来源比较多元，接口相对杂乱，大家在使用用过程中需注意检查数据的准确性。

接下来，罗列一些常用的数据接口，供参考使用。

获取A 股上市公司实时行情数据
该数据来源新浪，使用时候注意频率，不然会被封IP

In [ ]:
import akshare as ak
stock_zh_a_spot_df = ak.stock_zh_a_spot()
stock_zh_a_spot_df

获取某股票历史行情数据
以浦发银行（sh600000）为例：

In [ ]:
import akshare as ak
stock_zh_a_daily_qfq_df = ak.stock_zh_a_daily(symbol="sh600000", adjust="hfq")
stock_zh_a_daily_qfq_df

获取公募开放式基金数据

In [ ]:
import akshare as ak
fund_em_open_fund_daily_df = ak.fund_em_open_fund_daily()
fund_em_open_fund_daily_df

上面简单列举了几个数据接口，AKshare还提供了大量的股票、基金、期货等数据，大家可以去探索下。